### Я открыл ноутбук

- а теперь, когда проект сдан, займемся его разработкой. Надо подтянуть руберта и сделать бинарную классификацию. Возьмем за хорошие отзывы рейтинга не менее 6.

In [23]:
!pip uninstall -y transformers accelerate
!pip install torch==2.1.2
!pip install transformers==4.38.2
!pip install accelerate==0.28.0
!pip install datasets==2.18.0
!pip install scikit-learn
!pip install "numpy<2"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Found existing installation: transformers 4.38.2
Uninstalling transformers-4.38.2:
  Successfully uninstalled transformers-4.38.2
Found existing installation: accelerate 0.28.0
Uninstalling accelerate-0.28.0:
  Successfully uninstalled accelerate-0.28.0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached transformers-4.38.2-py3-none-any.whl.metadata (130 kB)
Using cached transformers-4.38.2-py3-none-any.whl (8.5 MB)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached accelerate-0.28.0-py3-none-any.whl.metadata (18 kB)
Using cached accelerate-0.28.0-py3-none-any.whl (290 kB)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [24]:
import pandas as pd
import numpy as np
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          TrainingArguments, Trainer)
from sklearn.metrics import accuracy_score, f1_score, classification_report

- немного порежем данные чтобы отзывов с оценкой 5 стало в 5 раз меньше

In [25]:

df = pd.read_csv("lamoda_reviews.csv")
df.head(10)
df['Rating'].value_counts()

df_5_sampled = df[df["Rating"] == 5].sample(n=20000, random_state=42)


df_rest = df[df["Rating"] != 5]


df_new = pd.concat([df_5_sampled, df_rest], ignore_index=True)

df = df_new.sample(frac=1, random_state=42).reset_index(drop=True)

print(df["Rating"].value_counts())
print("Общий размер:", len(df))

Rating
5    20000
4     7315
3     3352
1     1425
2     1094
Name: count, dtype: int64
Общий размер: 33186


In [ ]:
df = df.dropna(subset=["Review"])
df["label"] = (df["Rating"] >= 4).astype(int) 
df = df[["Review", "label"]].rename(columns={"Review": "text"})


In [27]:
dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.2, seed=42)

checkpoint = "cointegrated/rubert-tiny"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=256)

tokenized_dataset = dataset.map(tokenize, batched=True)

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds)
    }

/Users/berdov/ml/basic_ml_hse/myenv2/lib/python3.11/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map: 100%|██████████| 6637/6637 [00:00<00:00, 13779.15 examples/s]
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:


training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    learning_rate=2e-5,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    logging_dir="./logs",
    report_to="none"
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()



/Users/berdov/ml/basic_ml_hse/myenv2/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
 13%|█▎        | 648/4980 [01:35<10:37,  6.79it/s]

In [ ]:
results = trainer.evaluate()
print("\nФинальные метрики на тесте:")
for k, v in results.items():
    print(f"{k}: {v:.4f}")


predictions = trainer.predict(tokenized_dataset["test"])
y_true = predictions.label_ids
y_pred = np.argmax(predictions.predictions, axis=1)

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=["нет", "да"]))


,text,label
0,"Отличные кроссовки, очень лёгкие, на ножке акк...",1
1,"Красивые, стильные, удобные.",1
2,Все хорошо. Идет в размер,1
3,"Удобные и красивые, но вся грязь собирается на...",1
4,Удобные пока не носоли.,1
5,"Отличные кроссовки, пришли быстро. Мягкие и уд...",1
6,Хорошие кроссовки для Весна-Лето,1
7,"Отличные кроссовки, на вид хорошего качества. ...",1
8,Все супер,1
9,норм,1
